In [0]:
def bronze_():
    df = spark.read.format("jdbc") \
        .option("url", "jdbc:sqlserver://sql.database.windows.net;databaseName=database") \
        .option("username", "x") \
        .option("password", "x") \
        .option("dbtable", "cdc.SalesLT_Customer_CT") \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .load()
    return (df)

In [0]:
%sql
USE CATALOG content_job;
USE SCHEMA bronze;

In [0]:
df = spark.table("silver.silver_account_details")
df.display()

In [0]:
%sql
CREATE SCHEMA content_job.temp

In [0]:
%sql
--DROP TABLE content_job.silver.silver_account_details

In [0]:
def bronze_account_user():
    return (
        spark.read
        .format("jdbc") 
        .option("url", dbutils.secrets.get(scope="sm-secret-scope", key = "social-media-project-db-jdbc")) 
        .option("username", dbutils.secrets.get(scope="sm-secret-scope", key = "social-media-project-dblog")) 
        .option("password", dbutils.secrets.get(scope="sm-secret-scope", key = "social-media-project-secret")) 
        .option("dbtable", dbutils.secrets.get(scope="sm-secret-scope", key = "social-media-project-db-tab-acc-users")) 
        .load()
        .display()
    )

In [0]:
%sql
--DELETE FROM content_job.bronze.bronze_account_user

In [0]:
%sql
DESCRIBE DETAIL content_job.bronze.bronze_account_user

In [0]:
display(dbutils.fs.ls("/mnt/content_job/bronze/bronze_account_user"))

In [0]:
%sql
SELECT * FROM content_job.silver.silver_account_details

In [0]:
%sql
--TRUNCATE TABLE content_job.silver.account_user_scd_type2

In [0]:
%sql
--UPDATE content_job.bronze.bronze_account_user SET is_group = false WHERE account_id = 1379